In [ ]:
#Import základných knižníc
from mylibraries import *

#Načítanie datasetu do premennej df
df = pd.read_csv('UCI_Credit_Card.csv')

#Nahradenie chýbajúcich hodnôt pomocou metódy fillna
df.fillna(df.mean(), inplace=True)  # Nahradíme chýbajúce hodnoty priemerom

#Premenovanie atributu na kratsi nazov
df = df.rename(columns={'default.payment.next.month': 'def_pay'})

#Premenovanie atributu PAY_0 na PAY_1
df = df.rename(columns={'PAY_0': 'PAY_1'})

#Spojime hodnoty vzdelania 0, 5 a 6 do kategorie 4: other
df['EDUCATION'] = df['EDUCATION'].replace({0: 4, 5: 4, 6: 4})

#Spojime hodnoty marriage z 0 do kategorie 3: other
df['MARRIAGE'] = df['MARRIAGE'].replace({0: 3})

#Premenovanie atributu PAY_0 na PAY_1
df = df.rename(columns={'PAY_0': 'PAY_1'})

df = df.drop(columns=['ID'])

#Label-encoding pre atribut SEX
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
df['SEX'] = label_encoder.fit_transform(df['SEX'])

# %%
pd.set_option('display.max_columns', 25)
df.head(5)


In [ ]:

#vytvorim si novu premennu pre vsetky platby a pre nesplatene platby
df['pays'] = df['PAY_1'] + df['PAY_2'] + df['PAY_3'] + df['PAY_4'] + df['PAY_5'] + df['PAY_6']

#vytvorim si novu premennu pre vsetky hodnoty amounts
df['bills'] = df['BILL_AMT1'] + df['BILL_AMT2'] + df['BILL_AMT3'] + df['BILL_AMT4'] + df['BILL_AMT5'] + df['BILL_AMT6'] - df['PAY_AMT1'] + df['PAY_AMT2'] + df['PAY_AMT3'] + df['PAY_AMT4'] + df['PAY_AMT5'] + df['PAY_AMT6']


#aplikujeme one-hot encoding na education a marriage
df = pd.get_dummies(df, columns=['EDUCATION', 'MARRIAGE'])

#odstranime atributy education_ine a marriage_ine
df.drop(['EDUCATION_3','MARRIAGE_3'],axis = 1, inplace = True)

#one-hot encoding na atributy pay_1-pay_6
df = pd.get_dummies(df, columns = ['PAY_1',	'PAY_2',	'PAY_3',	'PAY_4',	'PAY_5',	'PAY_6'], drop_first = True )

In [ ]:
df.head()

In [ ]:
X = df.drop(columns=['def_pay']) 
y = df['def_pay']  

# Rozdelenie dát na trénovaciu a testovaciu množinu
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Rozdelenie dát na trénovaciu a testovaciu množinu
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Použitie SMOTE na vyváženie trénovacej množiny
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Štandardizácia dát
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_smote)
X_test_scaled = scaler.transform(X_test)


In [ ]:
# Prevod numpy.ndarray na pandas.DataFrame
X_train_df = pd.DataFrame(X_train_smote, columns=X.columns)
X_test_df = pd.DataFrame(X_test, columns=X.columns)
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns)
# Prevod y_train a y_test na pandas.Series
y_train = pd.Series(y_train_smote.values.ravel(), name='def_pay')
y_test = pd.Series(y_test.values.ravel(), name='def_pay')


# Uloženie do CSV súboru
X_train_df.to_csv('X_train.csv', index=False)
X_test_df.to_csv('X_test.csv', index=False)
X_train_scaled_df.to_csv('X_train_scaled.csv', index=False)
X_test_scaled_df.to_csv('X_test_scaled.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)